In [26]:
from google.colab import drive

import csv
import pandas as pd
import ast
import re

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
base_path = '/content/drive/MyDrive/빅데이터 처리/팀플/Big Data Processing'
# 경로는 본인 환경 맞게 수정

In [30]:
combined_path = base_path + '/data/combined_data_for_similar_game_finder_supported_languages.csv'

In [31]:
# with open(kaggle_path, 'r', encoding='cp949', errors='replace') as kaggle_file, \
# open(steam_1_path, 'r', encoding='utf8', errors='replace') as steam_1_file, \
# open(steam_2_path, 'r', encoding='utf8', errors='replace') as steam_2_file, \
# open(vg_1_path, 'r', encoding='cp949', errors='replace') as vg_1_file, \
# open(vg_2_path, 'r', encoding='cp949', errors='replace') as vg_2_file, \
with open(combined_path, 'r', encoding='cp949', errors='replace') as combined_file:
#     kaggle_df = pd.read_csv(kaggle_file)
    # steam_1_df = pd.read_csv(steam_1_file)
    # steam_2_df = pd.read_csv(steam_2_file)
#     vg_1_df = pd.read_csv(vg_1_file)
#     vg_2_df = pd.read_csv(vg_2_file)
    combined_df = pd.read_csv(combined_file)

In [32]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58851 entries, 0 to 58850
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   AppID                58851 non-null  int64 
 1   Name                 58840 non-null  object
 2   Required age         58851 non-null  int64 
 3   Supported languages  58851 non-null  int64 
 4   Categories           58170 non-null  object
 5   Genres               58789 non-null  object
 6   Tags                 50823 non-null  object
 7   Overall Reviews      52548 non-null  object
 8   Currently popular    58851 non-null  object
 9   Game Description     55172 non-null  object
dtypes: int64(3), object(7)
memory usage: 4.5+ MB


In [33]:
# steam_1_df = steam_1_df.iloc[:10, 5:]
# steam_2_df = steam_2_df.iloc[:, 5:]
# vg_1_df = vg_1_df.iloc[:, 3:]
# vg_2_df = vg_2_df.iloc[1:, 3:]
combined_df_1 = combined_df.iloc[:, [8, 9]]

In [34]:
# print(steam_1_df)
print(combined_df_1)

                                       Currently popular  \
0      ['Not Recommended, \n\t\t\t\t\tAt first glance...   
1      ["Recommended, \n\t\t\t\t\tOne Finger Death Pu...   
2                                                     []   
3      ['Recommended, \nIntroductionWhile 2D platform...   
4      ["Not Recommended, \n\t\t\t\t\tSo, this was on...   
...                                                  ...   
58846  ['Recommended, \n\t\t\t\t\tA fun cute games fo...   
58847  ['Recommended, \n\t\t\t\t\tthis game has no pl...   
58848  ["Recommended, \n\t\t\t\t\tThis game was amazi...   
58849                                                 []   
58850                                                 []   

                                        Game Description  
0      Galactic Bowling is an exaggerated and stylize...  
1      THE LAW!!Looks to be a showdown atop a train. ...  
2      Jolt Project:The army now has a new robotics p...  
3      In bizarre lands, play as a viscous 

In [35]:
def parse_and_join_list(s, delimiter=', '):
    if isinstance(s, str) and s.startswith('[') and s.endswith(']'):
        try:
            lst = ast.literal_eval(s)
            if isinstance(lst, list):
                cleaned_lst = [elem.replace('Not Recommended, ', '').replace('Recommended, ', '') for elem in lst]
                return delimiter.join(cleaned_lst)
        except (ValueError, SyntaxError):
            pass
    return s


combined_df_1['Currently popular'] = combined_df_1['Currently popular'].apply(parse_and_join_list)


<ipython-input-35-a53588bb27ad>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df_1['Currently popular'] = combined_df_1['Currently popular'].apply(parse_and_join_list)


In [36]:
combined_df_1['Currently popular'] = combined_df_1['Currently popular'].apply(parse_and_join_list)
print(combined_df_1)

                                       Currently popular  \
0      \n\t\t\t\t\tAt first glance, Galactic Bowling ...   
1      \n\t\t\t\t\tOne Finger Death PunchBut it's ado...   
2                                                          
3      \nIntroductionWhile 2D platformer and puzzle g...   
4      \n\t\t\t\t\tSo, this was one of the Steam Next...   
...                                                  ...   
58846  \n\t\t\t\t\tA fun cute games for friends and f...   
58847  \n\t\t\t\t\tthis game has no players please ma...   
58848  \n\t\t\t\t\tThis game was amazing. The archite...   
58849                                                      
58850                                                      

                                        Game Description  
0      Galactic Bowling is an exaggerated and stylize...  
1      THE LAW!!Looks to be a showdown atop a train. ...  
2      Jolt Project:The army now has a new robotics p...  
3      In bizarre lands, play as a viscous 

<ipython-input-36-cf66faad13db>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df_1['Currently popular'] = combined_df_1['Currently popular'].apply(parse_and_join_list)


In [37]:
def remove_escape_sequences(s):
    if isinstance(s, str):
        return s.replace('\n', ' ').replace('\t', ' ')
    else:
        return s

In [38]:
string_columns = combined_df_1.select_dtypes(include='object').columns
combined_df_1[string_columns] = combined_df_1[string_columns].applymap(remove_escape_sequences)

print(combined_df_1)

                                       Currently popular  \
0            At first glance, Galactic Bowling appear...   
1            One Finger Death PunchBut it's adorable!...   
2                                                          
3       IntroductionWhile 2D platformer and puzzle ga...   
4            So, this was one of the Steam Next Fest ...   
...                                                  ...   
58846        A fun cute games for friends and family....   
58847        this game has no players please make it ...   
58848        This game was amazing. The architecture ...   
58849                                                      
58850                                                      

                                        Game Description  
0      Galactic Bowling is an exaggerated and stylize...  
1      THE LAW!!Looks to be a showdown atop a train. ...  
2      Jolt Project:The army now has a new robotics p...  
3      In bizarre lands, play as a viscous 

<ipython-input-38-74a0cdc0944e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df_1[string_columns] = combined_df_1[string_columns].applymap(remove_escape_sequences)


In [39]:
sample_text_data_path = base_path + '/data/sample_text_data.csv'

In [40]:
combined_df_1.to_csv(sample_text_data_path, index = False, encoding = 'cp949')